In [1]:
import openai
import os
from flask import Flask, request, jsonify
from flask_socketio import SocketIO, emit

In [ ]:
# Set up the OpenAI API key (ensure to set your API key as an environment variable)
openai.api_key = os.getenv("OPENAI_API_KEY")

# Create a Flask app
app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*", async_mode="gevent")

# Define a socket route for chatbot interaction
@socketio.on('chat_message')
def handle_chat_message(message):
    # Convert the message to a prompt for ChatGPT
    prompt = (
        "You are an assistant that provides answers to patient billing queries. "
        "Answer questions about billing data, overcharges, and other related concerns.\n"
        f"User: {message}\nAssistant:"
    )
    
    # Call the OpenAI API
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=150
    )
    
    # Get the chatbot response
    response_text = response.choices[0].text.strip()
    
    # Emit the response back to the client
    emit('chat_response', {'response': response_text})

# Run the Flask app
if __name__ == '__main__':
    socketio.run(app, host='0.0.0.0', port=5002, debug=True)